Imports and secrets

In [1]:
import csv
import requests
from dotenv import load_dotenv
import os
from datetime import date

load_dotenv()
NOTION_API = os.getenv('NOTION_API')
NOTION_DATABASE_ID = os.getenv('NOTION_DATABASE_ID')

Collect entries from Notion DB

In [2]:
data = []

headers = {
    'Authorization': f"Bearer {NOTION_API}",
    'Content-Type': 'application/json',
    'Notion-Version': '2022-06-28',
}

not_done = True
start_cursor = ''
while (not_done):
    body = {}
    if start_cursor != '':
        body = {
            'start_cursor': start_cursor
        }

    response = requests.post(f"https://api.notion.com/v1/databases/{NOTION_DATABASE_ID}/query", headers=headers, json=body)
    if response.status_code != 404:
        json = response.json()

        for row in json['results']:
            props = row['properties']
            name = props['Name']['title'][0]['plain_text']
            height = props['Height (cm)']['number']
            categories = f"[{','.join(list(map(lambda x: x['name'], props['Tags']['multi_select'])))}]"
            sources = f"[{','.join(list(map(lambda x: x['name'], props['Source']['multi_select'])))}]"

            data.append([name, height, categories, sources])

        if json['has_more']:
            start_cursor = json['next_cursor']
        else:
            not_done = False

print(data)

[['Dua Lipa', 173, '[Music]', '[Celeb Heights]'], ['Jenna Ortega', 151, '[Acting]', '[Celeb Heights]'], ['Jenna Fischer', 165, '[Acting]', '[Celeb Heights]'], ['Dev', 171, '[Other]', '[Us]'], ['Pablo Picasso', None, '[Historical]', '[]'], ['Fidel Castro', 185, '[Historical]', '[Celeb Heights]'], ['Nelson Mandela', 184, '[Historical]', '[Celeb Heights]'], ['Genghis Khan', None, '[Historical]', '[]'], ['Hayao Miyazaki', None, '[Writers]', '[]'], ['Che Guevara', None, '[Historical]', '[]'], ['Jimi Hendrix', 178, '[Music]', '[Celeb Heights]'], ['Marilyn Monroe', 166, '[Acting]', '[Celeb Heights]'], ['Aristotle', None, '[Historical]', '[]'], ['Marco Polo', None, '[Historical]', '[]'], ['Harry Styles', 178, '[Music]', '[Celeb Heights]'], ['Margot Robbie', 166, '[Acting]', '[Celeb Heights]'], ['Jennifer Lawrence', 171, '[Acting]', '[Celeb Heights]'], ['Taylor Swift', 176, '[Music]', '[Celeb Heights]'], ['Madonna', 163, '[Music]', '[Celeb Heights]'], ['Michael Phelps', 189, '[Sports]', '[Celeb

Save entries to csv

In [3]:
# only save entries if they have a height
save_if_have_height: bool = True


# filter entries if needed (only get if they have a height)
if save_if_have_height:
    new_data = []
    for entry in data:
        if entry[1] != None:
            new_data.append(entry)

    data = new_data

# save to a dated csv
with open(f"data/notion-data-{date.today()}.csv", 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    
    writer.writerow(['Name', 'Height', 'Category', 'Source'])   
    writer.writerows(data)